In [1]:
# from google.colab import drive
# drive.mount('/gdrive')

In [2]:
import json
import numpy as np
import pickle
import pandas as pd
from datetime import datetime
import pytz
import time

In [3]:
# When working with personal drive
# model_file_path = '/gdrive/My Drive/trip_model.pkl'
# scaler_file_path = '/gdrive/My Drive/scaler_model.pkl'
# data_file_path = '/gdrive/My Drive/station_data.json'
# output_file_path_pred = '/gdrive/My Drive/station_data.json'
# output_file_path_pandas = "/gdrive/My Drive/output_priority.json"

In [4]:
# model_file_path = '/gdrive/My Drive/Citiq/trip_model.pkl'
# scaler_file_path = '/gdrive/My Drive/Citiq/scaler_model.pkl'
# data_file_path = '/gdrive/My Drive/Citiq/station_data.json'
# output_file_path_pred = '/gdrive/My Drive/Citiq/station_data.json'
# output_file_path_pandas = "/gdrive/My Drive/Citiq/output_priority.json"

In [5]:
model_file_path = 'adb-model.pkl'
scaler_file_path = 'scaler_model.pkl'
data_file_path = 'station_data.json'
output_file_path_pred = 'station_data.json'
output_file_path_pandas = "output_priority.json"

In [6]:
def get_current_datetime():
    eastern = pytz.timezone('US/Eastern')
    return datetime.now(eastern).strftime("%Y-%m-%d %H:%M:%S")
eastern = pytz.timezone('US/Eastern')

In [7]:
def load_model(file_path):
    with open(file_path, 'rb') as model_file:
        return pickle.load(model_file)

# def load_scaler(file_path):
#     with open(file_path, 'rb') as scaler_file:
#         return pickle.load(scaler_file)

def load_station_data(file_path):
    with open(file_path, 'r') as data_file:
        return json.load(data_file)

# def scale_features(data_point, scaler):
#     return scaler.transform([data_point])[0]

# def predict_traffic(model, scaled_data_point):
#     return model.predict([scaled_data_point])[0]


def add_predictions_to_data(station_data, model):
    # Convert the station_data list of dictionaries to a pandas DataFrame
    df = pd.DataFrame(station_data)

    # Extract features into a separate DataFrame and rename columns
    features = df[['day_of_week', 'hour', 'local_id']]
    features = features.rename(columns={'day_of_week': 'day', 'hour': 'hour', 'local_id': 'station_id'})

    # Scale the features using the loaded scaler
    # scaled_features = scaler.transform(features)
    features["day"] /= 7
    features["hour"] /= 24
    
    # Use the trained model to predict the target variable
    df['predicted_traffic'] = model.predict(features)
    return df['predicted_traffic']


In [8]:
rf_model = load_model(model_file_path)
station_data = load_station_data(data_file_path)

In [9]:
# def runPrediction():
#     rf_model = load_model(model_file_path)
#     #scaler = load_scaler(scaler_file_path)
#     station_data = load_station_data(data_file_path)

#     add_predictions_to_data(station_data, rf_model)
    

In [10]:
latest_predictions = add_predictions_to_data(station_data,rf_model)

C:\Users\treha\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but AdaBoostRegressor was fitted without feature names
  warnings.warn(


In [11]:
current_hour_global = datetime.now(eastern).hour
print(current_hour_global)

10


In [12]:
def process_batch(current_hour_global,latest_predictions):
    # Read the JSON file into a Pandas DataFrame
    df = pd.read_json(output_file_path_pred)

    # Create a new column maintenance_priority
    df["maintenance_priority"] = (df["num_docks_disabled"] + df["num_vehicles_disabled"]) / \
                                 (df["num_docks_available"] + df["num_vehicles_available"] + 1)
    current_datetime = datetime.now(eastern)
    if current_datetime.hour != current_hour_global:
        latest_predictions = add_predictions_to_data(station_data,rf_model)
        current_hour_global = current_datetime.hour
    df["predicted_traffic"] = latest_predictions
    # Order the DataFrame in descending order of maintenance_priority
    df = df.sort_values(by="maintenance_priority", ascending=False)

    df.to_json(output_file_path_pandas, orient="records", lines=True)

In [ ]:
# Set the interval in seconds
interval_seconds = 5

retry_delay_seconds = 10
try:
    # Run indefinitely
    while True:
        try:
            # Record the start time for each iteration
            iteration_start_time = time.time()
    
            # Process the batch
            process_batch(current_hour_global,latest_predictions)
    
            # Calculate and print the time taken for the iteration
            iteration_end_time = time.time()
            iteration_elapsed_time = iteration_end_time - iteration_start_time
            print(f"Time taken for iteration: {iteration_elapsed_time:.2f} seconds")
    
            # Wait for the specified interval
            time.sleep(interval_seconds)

        except ValueError as ve:
            # Handle ValueError
            print(f"Error: {ve}")
            print(f"Retrying in {retry_delay_seconds} seconds...")
            time.sleep(retry_delay_seconds)
            continue

except KeyboardInterrupt:
    # Handle keyboard interrupt (e.g., press Ctrl+C to stop the loop)
    print("Stopping the application")


Time taken for iteration: 0.04 seconds
Time taken for iteration: 0.03 seconds
Time taken for iteration: 0.06 seconds
Time taken for iteration: 0.03 seconds
Time taken for iteration: 0.04 seconds
Time taken for iteration: 0.03 seconds
Time taken for iteration: 0.04 seconds
Time taken for iteration: 0.03 seconds
Time taken for iteration: 0.04 seconds
Time taken for iteration: 0.04 seconds
Time taken for iteration: 0.03 seconds
Time taken for iteration: 0.03 seconds
Time taken for iteration: 0.04 seconds
Time taken for iteration: 0.03 seconds
Time taken for iteration: 0.03 seconds
Time taken for iteration: 0.03 seconds
Time taken for iteration: 0.04 seconds
Time taken for iteration: 0.03 seconds
Time taken for iteration: 0.04 seconds
Time taken for iteration: 0.03 seconds
Time taken for iteration: 0.04 seconds
Time taken for iteration: 0.03 seconds
Time taken for iteration: 0.03 seconds
Time taken for iteration: 0.04 seconds
Time taken for iteration: 0.03 seconds
Time taken for iteration:

C:\Users\treha\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but AdaBoostRegressor was fitted without feature names
  warnings.warn(


Time taken for iteration: 0.42 seconds


C:\Users\treha\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but AdaBoostRegressor was fitted without feature names
  warnings.warn(


Time taken for iteration: 0.16 seconds


...

Time taken for iteration: 0.14 seconds


C:\Users\treha\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but AdaBoostRegressor was fitted without feature names
  warnings.warn(


Time taken for iteration: 0.14 seconds


C:\Users\treha\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but AdaBoostRegressor was fitted without feature names
  warnings.warn(


Time taken for iteration: 0.13 seconds
